In [20]:
import pandas as pd
import pickle
from itertools import compress

In [2]:
with open('data/csgo_df.pkl', 'rb') as f:
    df_raw = pickle.load(f)

In [3]:
player_count = pd.read_csv('data/csgo.csv')

In [4]:
df = df_raw.copy()

#### List of items:

In [5]:
items = df.item_name.unique()

#### Possible parentheticals:

In [16]:
parentheticals = set([item[item.rfind('('):] for item in items if item.find('(') >= 0])
parentheticals

{'(Battle Green)',
 '(Battle-Scarred)',
 '(Bazooka Pink)',
 '(Blood Red)',
 '(Blue)',
 '(Brick Red)',
 '(Cash Green)',
 '(Desert Amber)',
 '(Dust Brown)',
 '(Factory New)',
 '(Field-Tested)',
 '(Foil)',
 '(Foil)  | MLG Columbus 2016',
 '(Foil) | Atlanta 2017',
 '(Foil) | Boston 2018',
 '(Foil) | Cluj-Napoca 2015',
 '(Foil) | Cologne 2015',
 '(Foil) | Cologne 2016',
 '(Foil) | DreamHack 2014',
 '(Foil) | Katowice 2014',
 '(Foil) | Katowice 2015',
 '(Foil) | Krakow 2017',
 '(Foil) | MLG Columbus 2016',
 '(Frog Green)',
 '(Gold) | Boston 2018',
 '(Gold) | Krakow 2017',
 '(Holo)',
 '(Holo) | Atlanta 2017',
 '(Holo) | Boston 2018',
 '(Holo) | Cologne 2014',
 '(Holo) | Cologne 2016',
 '(Holo) | DreamHack 2014',
 '(Holo) | Katowice 2014',
 '(Holo) | Katowice 2015',
 '(Holo) | Krakow 2017',
 '(Holo) | MLG Columbus 2016',
 '(Holo-Foil)',
 '(Jungle Green)',
 '(Minimal Wear)',
 '(Monarch Blue)',
 '(Monster Purple)',
 '(Princess Pink)',
 '(Red)',
 '(SWAT Blue)',
 '(Shark White)',
 '(Tiger Orange)'

#### We have colors, quality, foil, events (after '|', and likely separate of parentheticals if foil is missing)

In [22]:
list(compress(items, ['(Foil)  | MLG Columbus 2016' in item for item in items]))

['Sticker | AdreN (Foil)  | MLG Columbus 2016']

In [23]:
'Sticker | AdreN (Foil) | MLG Columbus 2016'

'Sticker | AdreN (Foil) | MLG Columbus 2016'

In [26]:
df = df.replace(to_replace='Sticker | AdreN (Foil)  | MLG Columbus 2016', value='Sticker | AdreN (Foil) | MLG Columbus 2016')

#### Removed the extra space in that string

#### Looking at the items without parentheses

In [31]:
df['parenthetical'] = [item.find('(') >= 0 for item in df.item_name]

In [36]:
len(df[~df.parenthetical].item_name.unique())

1450

1450 unique items left to categorize